In [2]:
# Import modules and read the original file
import pandas as pd
import os
path = os.path.join("Resources", "purchase_data.csv")
df = pd.read_csv(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
Purchase ID    780 non-null int64
SN             780 non-null object
Age            780 non-null int64
Gender         780 non-null object
Item ID        780 non-null int64
Item Name      780 non-null object
Price          780 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.7+ KB


In [3]:
# Player Count
players = df['SN'].nunique()
print("There are %s unique players in this dataset." % players)

There are 576 unique players in this dataset.


In [5]:
# Purchasing Analysis (Total)

items = df['Item ID'].nunique()
purchases = df['Purchase ID'].nunique()
revenue = df['Price'].sum()
avg_price = df['Price'].mean()

purchasingAnalysisTotal = pd.DataFrame({
    "Num of Items": [items],
    'Average Price': [avg_price],
    'Num of Purchases': [purchases],
    'Total Sales': [revenue],
})

purchasingAnalysisTotal['Average Price'] = purchasingAnalysisTotal['Average Price'].map('${:.2f}'.format)
purchasingAnalysisTotal['Total Sales'] = purchasingAnalysisTotal['Total Sales'].map('${:,.2f}'.format)

purchasingAnalysisTotal

,Num of Items,Average Price,Num of Purchases,Total Sales
0,183,$3.05,780,"$2,379.77"


In [6]:
#Gender Demographics

unique_playerDF = df.drop_duplicates('SN')
unique_playerDF['Gender'].value_counts()

numMale = (unique_playerDF['Gender'] == 'Male').sum()
numFemale = (unique_playerDF['Gender'] == 'Female').sum()
numOther = (unique_playerDF['Gender'] == 'Other / Non-Disclosed').sum()

genderDemographics = pd.DataFrame({
    'Total Count': [numMale, numFemale, numOther],
    'Percentage of Players': [(numMale/players)*100,(numFemale/players)*100, (numOther/players)*100],
}, ['Males', 'Females', 'Other / Non-Disclosed'])

genderDemographics['Percentage of Players'] = genderDemographics['Percentage of Players'].map('{:.2f}%'.format)

genderDemographics

,Total Count,Percentage of Players
Males,484,84.03%
Females,81,14.06%
Other / Non-Disclosed,11,1.91%


In [8]:
# Purchasing Analysis (Gender)

byGenderDF = df.groupby(['Gender'])

genderCount = byGenderDF['SN'].count()
genderMeanPrice = byGenderDF['Price'].mean()
genderTotal = byGenderDF['Price'].sum()
genderUnique = byGenderDF['SN'].nunique()
genderAvgPurch = (genderTotal/genderUnique)

purchasingAnalysisGender = pd.DataFrame({
    'Purchase Count': genderCount,
    'Average Purchase Price': genderMeanPrice.map('${:,.2f}'.format),
    'Total Purchase Sum': genderTotal.map('${:,.2f}'.format),
    'Avg Total Purchase per Person': genderAvgPurch.map('${:,.2f}'.format)},
    ['Female', 'Male', 'Other / Non-Disclosed'])

purchasingAnalysisGender.index.name = 'Gender'

purchasingAnalysisGender

,Purchase Count,Average Purchase Price,Total Purchase Sum,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [10]:
# Age Demographics

bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
label_list = ["0-10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_groups = pd.cut(unique_playerDF['Age'], bins, labels = label_list)

ageDemographicsDF = pd.DataFrame({
    'Total Count': age_groups.value_counts().sort_index()
}, label_list)

ageDemographicsDF['Percentage of Players'] = ((ageDemographicsDF['Total Count']/players)*100).map('{:,.2f}%'.format)

ageDemographicsDF

,Total Count,Percentage of Players
0-10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [11]:
# Purchasing Analysis (Age)

df['Age Group'] = pd.cut(df['Age'], bins, labels = label_list)
byAgeGroupAll = df.groupby(['Age Group'])

ageUnique = byAgeGroupAll['SN'].nunique()
purchase_count = byAgeGroupAll['Purchase ID'].count()
avg_purchase_price = byAgeGroupAll['Price'].mean()
total_Value_by_age = byAgeGroupAll['Price'].sum()
avg_Total_purchase = (total_Value_by_age/ageUnique)

purchasingAnalysisAge = pd.DataFrame({
    'Purchase Count':purchase_count,
    'Average Purchase Price': avg_purchase_price.map('${:,.2f}'.format),
    'Total Purchase Value': total_Value_by_age.map('${:,.2f}'.format),
    'Avg Total Purchase per Person': avg_Total_purchase.map('${:,.2f}'.format),
}, label_list)
purchasingAnalysisAge.index.name = 'Age Groups'

purchasingAnalysisAge

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Groups,,,,
0-10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [12]:
# Most Popular Items

itemDF = df[['Item ID', 'Item Name', 'Price']]
groupByItem = itemDF.groupby(['Item ID', 'Item Name'])

purchase_count = groupByItem['Price'].count()
purchase_total = groupByItem['Price'].sum()
item_price = (purchase_total/purchase_count)

df2 = pd.DataFrame({
    'Purchase Count': groupByItem['Price'].count(),
})
df2['Item Price'] = item_price
df2['Total Purchase Value'] = purchase_total

#copy of unformatted df2 to use for the next section
df3 = df2.copy()

df2['Item Price'] = item_price.map('${:,.2f}'.format)
df2['Total Purchase Value'] = purchase_total.map('${:,.2f}'.format)
df2.sort_values('Purchase Count', ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [13]:
# Most Profitable Items

df3 = df3.sort_values(by='Total Purchase Value', ascending=False).head()

df3['Item Price'] = df3['Item Price'].map('${:,.2f}'.format)
df3['Total Purchase Value'] = df3['Total Purchase Value'].map('${:,.2f}'.format)
df3.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
